### Imports

In [13]:
import nltk
import re
from collections import Counter
import string
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from nltk import bigrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm
import numpy as np
import math
import statistics

[nltk_data] Downloading package stopwords to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sun Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### import files

In [14]:
# import all the files in this block

data_frame_test = pd.read_csv("../../Data/Analysis_ready/clean_test_reddit.csv")
# data_frame_odi = pd.read_csv(None)
# data_frame_t20 = pd.read_csv(None)

In [15]:
data_frame_test.head()

,Post_id,Id,Time,Author,Score,No_comments,Body,Type,Pre-processed_body,Vader_Pre-processed_body
0,1f0syen,1f0syen,2024-08-25 20:04:21,NoQuestion4045,2445,281,Bangladesh beats Pakistan for the first ever i...,Post,bangladesh beat pakistan first ever test cricket,Bangladesh beats Pakistan for the first ever i...
1,1f0syen,lju4ld6,2024-08-25 20:07:28,deleted,1151,0,"Fort Rawalpindi, yet to be captured, by the ho...",Comment,fort rawalpindi yet capture home side,"Fort Rawalpindi , yet to be captured , by the ..."
2,1f0syen,lju9bzy,2024-08-25 20:58:53,mofucker20,289,0,That stat is so ridiculous. What’s even the us...,Comment,stat ridiculous whats even use home ground hav...,That stat is so ridiculous . What ’ s even the...
3,1f0syen,ljubwki,2024-08-25 21:24:17,SquiffyRae,173,0,Part of the problem is Pindi is such a highway...,Comment,part problem pindi highway road there's always...,Part of the problem is Pindi is such a highway...
4,1f0syen,ljvk09z,2024-08-26 02:17:08,deleted,48,0,It's particularly risky for a team like Pakist...,Comment,particularly risky team like pakistan tendency...,It's particularly risky for a team like Pakist...


# Graph: Interaction between formats for the most active members

### Identify the Top N most active members

In [16]:
# extract and sort comment count for non-deleted users

# test 
test_comment_count_by_author = data_frame_test.groupby('Author').size().reset_index(name='Comment/Post_Count')
test_comment_count_by_author = test_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
test_comment_count_by_author = test_comment_count_by_author[test_comment_count_by_author['Author'] != "deleted"]

# # odi
# odi_comment_count_by_author = data_frame_odi.groupby('Author').size().reset_index(name='Comment/Post_Count')
# odi_comment_count_by_author = odi_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
# odi_comment_count_by_author = odi_comment_count_by_author[odi_comment_count_by_author['Author'] != "deleted"]

# # t20
# t20_comment_count_by_author = data_frame_t20.groupby('Author').size().reset_index(name='Comment/Post_Count')
# t20_comment_count_by_author = t20_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
# t20_comment_count_by_author = t20_comment_count_by_author[t20_comment_count_by_author['Author'] != "deleted"]


In [17]:
head_count = 50

top_n_test_author_list = test_comment_count_by_author['Author'].head(head_count).to_list()
# top_n_odi_author_list  = odi_comment_count_by_author['Author'].head(head_count).to_list()
# top_n_t20_author_list  = t20_comment_count_by_author['Author'].head(head_count).to_list()

top_n_odi_author_list = []
top_n_t20_author_list = []


In [18]:
top_n_test_author_list[:5]

['VisRock',
 'abettertomorrow47',
 'Classymuch',
 'crashbandicoochy',
 'mongrelbifana']

### Filter the post and comments by the top users

In [19]:
# to prevent some users has barely drop off the top rank, I have included all top users in all top 3 group for filtering
top_n_users_set = set()
top_n_users_set.update(top_n_test_author_list)
top_n_users_set.update(top_n_odi_author_list)
top_n_users_set.update(top_n_t20_author_list)

to_n_author_content_test = data_frame_test[data_frame_test['Author'].isin(top_n_users_set)]
# to_n_author_content_odi  = data_frame_odi[data_frame_odi['Author'].isin(top_n_users_set)]
# to_n_author_content_t20  = data_frame_t20[data_frame_t20['Author'].isin(top_n_users_set)]

In [20]:
to_n_author_content_test.head()

,Post_id,Id,Time,Author,Score,No_comments,Body,Type,Pre-processed_body,Vader_Pre-processed_body
36,1f0syen,ljuphhc,2024-08-25 23:15:35,confused_brown_dude,5,0,I wasn’t ready for this 😭,Comment,wasnt ready,I wasn ’ t ready for this 😭
65,1f0syen,ljuppbl,2024-08-25 23:17:07,confused_brown_dude,30,0,"I can’t get past the idiot thing 🤣, why does i...",Comment,cant get past idiot thing hit hard,"I can ’ t get past the idiot thing 🤣 , why doe..."
72,1f0syen,ljuv8bq,2024-08-25 23:53:59,Medical_Turing_Test,12,0,As a Fawad Alam truther it's important to note...,Comment,fawad alam truther important note ss sensational,As a Fawad Alam truther it's important to note...
74,1f0syen,ljv3br3,2024-08-26 00:42:41,Medical_Turing_Test,10,0,Probably. But Fawad is ancient.\n\nUnlikely. S...,Comment,probably fawad ancient unlikely saim ayub scor...,Probably . But Fawad is ancient . Unlikely . S...
76,1f0syen,ljv8d1c,2024-08-26 01:11:04,Medical_Turing_Test,4,0,https://www.espncricinfo.com/series/quaid-e-az...,Comment,please stop embarrassing,Please stop embarrassing yourself


In [21]:
to_n_author_content_test.shape

(15105, 10)

In [22]:
# TODO: Remember to import networkx
import networkx as nx

# here are the three main format
test = "Test"
odi  = "ODI"
t20  = "T20"

activity_graph = nx.DiGraph()
activity_graph.add_node(test)
activity_graph.add_node(odi)
activity_graph.add_node(t20)

user_check_set = set()

# to_n_author_content_test
# acc = author comment count
# test:
acc_test_list = to_n_author_content_test.groupby('Author').size().reset_index(name='Comment/Post_Count')

for index, row in acc_test_list.iterrows():
    
    user = row['Author']
    if user not in user_check_set:
        user_check_set.add(user)
        activity_graph.add_node(user)
    
    activity_graph.add_edge(user, test, weight=row['Comment/Post_Count'])

# # ODI
# acc_odi_list = to_n_author_content_odi.groupby('Author').size().reset_index(name='Comment/Post_Count')

# for index, row in acc_odi_list.iterrows():
    
#     user = row['Author']
#     if user not in user_check_set:
#         user_check_set.add(user)
#         activity_graph.add_node(user)
    
#     activity_graph.add_edge(user, odi, weight=row['Comment/Post_Count'])

# # T20
# acc_t20_list = to_n_author_content_t20.groupby('Author').size().reset_index(name='Comment/Post_Count')

# for index, row in acc_t20_list.iterrows():
    
#     user = row['Author']
#     if user not in user_check_set:
#         user_check_set.add(user)
#         activity_graph.add_node(user)
    
#     activity_graph.add_edge(user, t20, weight=row['Comment/Post_Count'])


In [23]:
nx.readwrite.write_graphml(activity_graph, 'test_graph.graphml')

### Related community to the top N users

In [61]:
# additional praw library:
from redditClient import redditClient
import praw
import prawcore
import time

import traceback

In [62]:
head_count = 50

top_n_test_author_list = test_comment_count_by_author['Author'].head(head_count).to_list()
# top_n_odi_author_list  = odi_comment_count_by_author['Author'].head(head_count).to_list()
# top_n_t20_author_list  = t20_comment_count_by_author['Author'].head(head_count).to_list()

top_n_odi_author_list = []
top_n_t20_author_list = []

all_active_user_list = top_n_test_author_list + top_n_odi_author_list + top_n_t20_author_list

In [63]:
all_active_user_list

['VisRock',
 'abettertomorrow47',
 'Classymuch',
 'crashbandicoochy',
 'mongrelbifana',
 'FondantAggravating68',
 'MedicalJello2',
 'Benny4318',
 'NoExplanation6203',
 'Tern_Larvidae-2424',
 'Frod02000',
 'Wazflame',
 'Axel292',
 'JokesFromTheCrease',
 'the_maddest_kiwi',
 'TrollerThomas',
 'WayToTheDawn63',
 'rambo_zaki',
 'TheRealMarkChapman',
 'MiachealFaraday',
 'Outside_Error_7355',
 'SuperFaiz21',
 'BigV95',
 'Jack-sprAt1212',
 'sellyme',
 'dhun_mohan',
 'Upstairs-Farm7106',
 'niceguysdofinish1st',
 'Ok_Vegetable263',
 'Merovech_II',
 '_rickjames',
 'atbg1936',
 'Subject-Ordinary6922',
 'One_Acanthaceae_1163',
 'NoirPochette',
 'Nark_Narkins',
 'Ghostly_100',
 'NotAsOriginal',
 'confused_brown_dude',
 'maffzlel',
 'kiwirish',
 'mondognarly_',
 'snomanDS',
 'Rndomguytf',
 'NormalTraining5268',
 'GoabNZ',
 'Medical_Turing_Test',
 'scubadoobidoo',
 'Doc8176',
 'ViolatingBadgers']

In [64]:
# construct Reddit client
# please update the 2auth verification code if necessary
client = redditClient()

# load the subreddit weAreMusicMakers
subreddit = client.subreddit('Cricket')
print(subreddit)

Cricket


In [67]:
import praw.exceptions

failed_user_list = []

def auto_retry_get_user_community(user: str) -> dict[str, int]:

    user_subreddit_count = {}
    user_subreddit_set = set()

    while True:
        try:
            user_info = client.redditor(user)
            submissions = user_info.comments.new(limit=1000)
            for submission in submissions:
                subreddit = submission.subreddit
                if subreddit not in user_subreddit_set:
                    user_subreddit_set.add(subreddit)
                    user_subreddit_count[subreddit] = 0
                
                user_subreddit_count[subreddit] += 1
        except prawcore.exceptions.Forbidden as ex:
            print(f"Redditor {user} doesn't want their data exposed.")
            failed_user_list.append(user)
            return None
        except prawcore.exceptions.TooManyRequests as ex:
            print("Rate Limited... Wait for 20 seconds and try again")
            # time.sleep(20)
            continue
        break

    print(f"Redditor {user} are extracted")
    return user_subreddit_count

active_users_activities = {}

for user in all_active_user_list:
    subreddit_count = auto_retry_get_user_community(user)
    if subreddit_count is not None:
        active_users_activities[user] = subreddit_count



Redditor VisRock doesn't want their data exposed.
Redditor abettertomorrow47 are extracted
Redditor Classymuch are extracted
Redditor crashbandicoochy are extracted
Redditor mongrelbifana are extracted
Redditor FondantAggravating68 are extracted
Redditor MedicalJello2 are extracted
Redditor Benny4318 are extracted
Redditor NoExplanation6203 are extracted
Redditor Tern_Larvidae-2424 are extracted
Redditor Frod02000 are extracted
Redditor Wazflame are extracted
Redditor Axel292 are extracted
Redditor JokesFromTheCrease are extracted
Redditor the_maddest_kiwi doesn't want their data exposed.
Redditor TrollerThomas are extracted
Redditor WayToTheDawn63 are extracted
Redditor rambo_zaki are extracted
Redditor TheRealMarkChapman are extracted
Redditor MiachealFaraday are extracted
Redditor Outside_Error_7355 are extracted
Redditor SuperFaiz21 are extracted
Redditor BigV95 are extracted
Redditor Jack-sprAt1212 are extracted
Redditor sellyme are extracted
Redditor dhun_mohan are extracted
Redd

In [47]:
allSubmissionSet = set(allSubmissions)
allSubmissionSet

{Subreddit(display_name='Cricket'),
 Subreddit(display_name='CricketAus'),
 Subreddit(display_name='CricketBuddies'),
 Subreddit(display_name='EnoughMuskSpam'),
 Subreddit(display_name='IndiaCricket'),
 Subreddit(display_name='MumbaiIndians'),
 Subreddit(display_name='PakCricket'),
 Subreddit(display_name='WorldTestChampionship'),
 Subreddit(display_name='cricketworldcup'),
 Subreddit(display_name='interestingasfuck')}

In [85]:
# this tests for getting top 10 most active subreddit that the users was in.
# Once this works, I will write a function to apply all other users

count_sort_testing = active_users_activities.get('abettertomorrow47')
count_sort_testing = dict(sorted(count_sort_testing.items(), key=lambda item: item[1], reverse=True))
count_sort_testing_top_10 = {}

limit = 0 
for key, count in count_sort_testing.items():

    # Since we need to show that on presentation,
    # let me blind some explicit words
    subreddit_name = key.display_name
    subreddit_name = subreddit_name.replace("Porn", "P**n")
    subreddit_name = subreddit_name.replace("Ass", "A**")

    count_sort_testing_top_10[subreddit_name] = count
    if limit >= 10:
        break
    limit += 1

count_sort_testing_top_10

{'Cricket': 816,
 'srilanka': 75,
 'TheDeprogram': 47,
 'Asia_irl': 22,
 'BatmanArkham': 10,
 'MarxistCulture': 10,
 'Ben10': 8,
 'DeathrattleP**n': 3,
 'GoodA**Sub': 2,
 'batman': 2,
 'ThePenguin': 1}

In [103]:
def extract_top_10_most_active_subreddit_from_user(intput: dict) -> dict[str, int]:
    sort_dict = dict(sorted(intput.items(), key=lambda item: item[1], reverse=True))
    top_10_most_active_subreddit = {}

    limit = 0 
    for key, count in sort_dict.items():

        # Since we need to show that on presentation,
        # let me blind some explicit words
        subreddit_name = key.display_name
        subreddit_name = subreddit_name.replace("Porn", "P**n")
        subreddit_name = subreddit_name.replace("Ass", "A**")
        subreddit_name = f"r/{subreddit_name}"

        top_10_most_active_subreddit[subreddit_name] = count
        if limit >= 6:
            break
        limit += 1

    return top_10_most_active_subreddit

top_10_subreddit_per_user = {}

for redditor, count_dict in active_users_activities.items():
    top_10_most_active_sub_of_the_user = extract_top_10_most_active_subreddit_from_user(count_dict)
    top_10_subreddit_per_user[redditor] = top_10_most_active_sub_of_the_user

In [104]:
top_10_subreddit_per_user['Classymuch']

{'r/Cricket': 927,
 'r/Damnthatsinteresting': 21,
 'r/Monash': 13,
 'r/MadeMeSmile': 5,
 'r/computerscience': 4,
 'r/CricketBuddies': 4,
 'r/Unexpected': 3}

In [105]:
# plot a graph to show the correlation between users and subreddit, attempting to find 
# other related communities. Count of the subreddit will also be used as weight to reflect 
# how active other group they are in

comm_graph = nx.DiGraph()
comm_node_set = set()

for redditor, comm_dict in top_10_subreddit_per_user.items():
    comm_graph.add_node(redditor)

    for comm_name, comm_count in comm_dict.items():
        if comm_name not in comm_node_set:
            comm_node_set.add(comm_name)
            comm_graph.add_node(comm_name)
        
        if comm_name != 'r/Cricket':
            print (comm_name)
            comm_graph.add_edge(redditor, comm_name, weight=comm_count)

nx.readwrite.write_graphml(comm_graph, '../../Data/Graphs/testing_comm_graph.graphml')


r/srilanka
r/TheDeprogram
r/Asia_irl
r/BatmanArkham
r/MarxistCulture
r/Ben10
r/Damnthatsinteresting
r/Monash
r/MadeMeSmile
r/computerscience
r/CricketBuddies
r/Unexpected
r/rugbyunion
r/kpop
r/olympics
r/Patriots
r/nfl
r/nba
r/IndiaCricket
r/DeathrattleP**n
r/SAcricket
r/delhicapitals
r/olympics
r/CricketBuddies
r/SquaredCircle
r/Guyana
r/nba
r/Barbados
r/cocktails
r/AnythingGoesNews
r/soccer
r/TwoSentenceHorror
r/notinteresting
r/SAcricket
r/Awww
r/hypotheticalsituation
r/rugbyunion
r/newzealand
r/chch
r/allblacks
r/taskmaster
r/ufc
r/Gunners
r/nba
r/nfl
r/politics
r/soccer
r/Boxing
r/MMA
r/singularity
r/GeForceNOW
r/EnglandCricket
r/AskReddit
r/FortNiteBR
r/SAcricket
r/Costco
r/crispcricket
r/EnglandCricket
r/YoungSheldon
r/mylittlepony
r/thomasthetankengine
r/RoleReversal
r/DeathInParadiseBBC
r/Piratefolk
r/OnePiece
r/NonPoliticalTwitter
r/australia
r/BlackPeopleTwitter
r/mildlyinfuriating
r/reddevils
r/soccer
r/rugbyunion
r/shitposting
r/unpopularopinion
r/ufc
r/springboks
r/beatle